In [1]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.naive_bayes import MultinomialNB

In [2]:
# carregando dataset, adicionando os dados em um dataframe fltrando as para trabalhanr somente com a classe 0 e 1
iris = load_iris()
iris_data = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_data['class'] = iris.target
iris_data = iris_data[iris_data['class'].isin([0, 1])]

In [3]:
# Discretizando os atributos do Iris para usar MultinomialNB
'''Discretização: Os atributos do conjunto de dados Iris são contínuos, mas o modelo MultinomialNB espera atributos discretos. Por isso, a discretização é aplicada aos atributos utilizando a função pd.cut do Pandas.
Modelo MultinomialNB: O código agora utiliza o modelo MultinomialNB, que fornece o atributo feature_log_prob_ necessário para calcular as PI-explicações.'''
for col in iris_data.columns[:-1]: # iterando sobre todas as colunas exceto a coluna class
    iris_data[col] = pd.cut(iris_data[col], bins=3, labels=False, include_lowest=True, duplicates='drop')
    # iris_data[col] são os dados de cada coluna
    # bins=3 divide os dados em 3 bins (categorias)
    # 

In [4]:
X = iris_data[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
y = iris_data['class']
clf = MultinomialNB() # Usando MultinomialNB
clf.fit(X, y)

MultinomialNB()

In [5]:
def calculate_pi_explanation_loglinear(instance, clf, features):
    """Calcula uma PI-explicação de menor tamanho para uma instância utilizando um algoritmo log-linear.

    Args:
        instance: A instância para a qual se deseja calcular a PI-explicação.
        clf: O classificador Naive Bayes treinado.
        features: Nomes dos atributos do conjunto de dados.

    Returns:
        Uma lista de pares valor-atributo que constituem a PI-explicação.
    """

    # Calcula a probabilidade da classe prevista
    predicted_class = clf.predict([instance])[0]
    log_prob_predicted_class = clf.predict_log_proba([instance])[0][predicted_class]

    # Calcula a probabilidade da outra classe
    log_prob_other_class = clf.predict_log_proba([instance])[0][1 - predicted_class]

    # Calcula a diferença de probabilidade logarítmica para cada atributo
    delta_features = [(feature,
                       clf.feature_log_prob_[predicted_class][i] - clf.feature_log_prob_[1 - predicted_class][i])
                      for i, feature in enumerate(features)]

    # Ordena os atributos pela diferença de probabilidade logarítmica (maior primeiro)
    delta_features.sort(key=lambda x: x[1], reverse=True)

    # Seleciona os atributos que contribuem para a classe prevista
    pi_explanation = []
    threshold = log_prob_predicted_class - log_prob_other_class
    current_threshold = 0
    for feature, delta in delta_features:
        current_threshold += delta
        if current_threshold > threshold:
            pi_explanation.append((feature, instance[i]))
            break

    return pi_explanation

# Exemplo de uso
instance = iris_data.iloc[0].to_list()[:-1]  # Seleciona a primeira instância
pi_explanation = calculate_pi_explanation_loglinear(instance, clf, iris.feature_names)
print(f"PI-explicação Log-Linear para a instância {instance}: {pi_explanation}") 

PI-explicação Log-Linear para a instância [0, 1, 0, 0]: []


c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(
c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(
c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(


In [7]:
def enumerate_pi_explanations(instance, clf, features):
    """Enumera todas as PI-explicações para uma instância utilizando um algoritmo de atraso polinomial.

    Args:
        instance: A instância para a qual se deseja calcular as PI-explicações.
        clf: O classificador Naive Bayes treinado.
        features: Nomes dos atributos do conjunto de dados.

    Returns:
        Uma lista de todas as PI-explicações possíveis.
    """

    predicted_class = clf.predict([instance])[0]
    log_prob_predicted_class = clf.predict_log_proba([instance])[0][predicted_class]
    log_prob_other_class = clf.predict_log_proba([instance])[0][1 - predicted_class]

    delta_features = [(feature,
                       clf.feature_log_prob_[predicted_class][i] - clf.feature_log_prob_[1 - predicted_class][i])
                      for i, feature in enumerate(features)]
    delta_features.sort(key=lambda x: x[1], reverse=True)

    pi_explanations = []
    def backtrack(idx, current_explanation, current_threshold):
        if current_threshold > log_prob_predicted_class - log_prob_other_class:
            pi_explanations.append(current_explanation)
            return

        if idx >= len(delta_features):
            return

        feature, delta = delta_features[idx]
        current_explanation.append((feature, instance[idx]))
        backtrack(idx + 1, current_explanation.copy(), current_threshold + delta)
        current_explanation.pop()
        backtrack(idx + 1, current_explanation.copy(), current_threshold)

    backtrack(0, [], 0)

    return pi_explanations

# Exemplo de uso
instance = iris_data.iloc[1].to_list()[:-1]  # Seleciona a segunda instância
pi_explanations = enumerate_pi_explanations(instance, clf, iris.feature_names)
print(f"PI-explicações com atraso polinomial para a instância {instance}: {pi_explanations}")

PI-explicações com atraso polinomial para a instância [0, 1, 0, 0]: []


c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(
c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(
c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(


o código não retornou nada

In [15]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carregando e preparando o dataset Iris com duas classes
iris = load_iris()
iris_data = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_data['class'] = iris.target
iris_data = iris_data[iris_data['class'].isin([0, 1])]  # Mantém apenas as classes 0 e 1

# Separando os dados de treino e teste
X = iris_data[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
y = iris_data['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinando o modelo de Regressão Logística
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

def calculate_pi_explanation_loglinear(instance, clf, features):
    """Calcula uma PI-explicação de menor tamanho para uma instância utilizando um algoritmo log-linear.

    Args:
        instance: A instância para a qual se deseja calcular a PI-explicação (array NumPy de uma única linha).
        clf: O classificador de Regressão Logística treinado.
        features: Nomes dos atributos do conjunto de dados.

    Returns:
        Uma lista de pares valor-atributo que constituem a PI-explicação.
    """

    # Calcula a probabilidade da classe prevista
    predicted_class = clf.predict(instance.reshape(1, -1))[0]
    prob_predicted_class = clf.predict_proba(instance.reshape(1, -1))[0][predicted_class]

    # Calcula a probabilidade da outra classe
    prob_other_class = clf.predict_proba(instance.reshape(1, -1))[0][1 - predicted_class]

    # Calcula a diferença de probabilidade para cada atributo
    delta_features = [(feature,
                       abs(clf.coef_[0][i] * instance[i]))
                      for i, feature in enumerate(features)]

    # Ordena os atributos pela diferença de probabilidade (maior primeiro)
    delta_features.sort(key=lambda x: x[1], reverse=True)

    # Seleciona os atributos que contribuem para a classe prevista
    pi_explanation = []
    threshold = prob_predicted_class - prob_other_class
    current_threshold = 0
    for idx, (feature, delta) in enumerate(delta_features): 
        current_threshold += delta
        pi_explanation.append((feature, instance[features.index(feature)]))
        if current_threshold > threshold:
            break

    return pi_explanation

# Selecionando a primeira instância do conjunto de teste como um array NumPy
instance = X_test.iloc[0].to_numpy()

# Calculando a PI-explicação para a instância
pi_explanation = calculate_pi_explanation_loglinear(instance, clf, list(X.columns))

# Exibindo a PI-explicação
print(f"PI-explicação Log-Linear para a instância {instance}: {pi_explanation}")


PI-explicação Log-Linear para a instância [6.  2.7 5.1 1.6]: [('petal length (cm)', 5.1)]


c:\Users\gleilson.pedro.SAUDE\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\gleilson.pedro.SAUDE\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
c:\Users\gleilson.pedro.SAUDE\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(


6.0: Comprimento da sépala (sepal length) em centímetros.
2.7: Largura da sépala (sepal width) em centímetros.
5.1: Comprimento da pétala (petal length) em centímetros.
1.6: Largura da pétala (petal width) em centímetros.

A PI-explicação sugere que o comprimento da pétala é o fator mais influente para a decisão do modelo sobre essa instância específica com 5.1 cm de comprimento.
Esse tipo de explicação ajuda a entender quais características são mais importantes para o modelo ao fazer uma previsão específica.

In [21]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carregando e preparando o dataset Iris com duas classes
iris = load_iris()
iris_data = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_data['class'] = iris.target
iris_data = iris_data[iris_data['class'].isin([2, 0])]  # Mudança para classes 1 e 2

# Separando os dados de treino e teste
X = iris_data[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
y = iris_data['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinando o modelo de Regressão Logística
clf = LogisticRegression(random_state=42)
clf.fit(X_train, y_train)

def calculate_pi_explanation_loglinear(instance, clf, features):
    """Calcula uma PI-explicação de menor tamanho para uma instância utilizando um algoritmo log-linear.

    Args:
        instance: A instância para a qual se deseja calcular a PI-explicação (array NumPy de uma única linha).
        clf: O classificador de Regressão Logística treinado.
        features: Nomes dos atributos do conjunto de dados.

    Returns:
        Uma lista de pares valor-atributo que constituem a PI-explicação.
    """

    # Calcula a probabilidade das duas classes previstas
    prob_predicted_class = clf.predict_proba(instance.reshape(1, -1))[0]

    # Calcula a diferença de probabilidade para cada atributo
    delta_features = []
    for i, feature in enumerate(features):
        coef = clf.coef_[0][i]  # Coeficiente do modelo para o atributo atual
        value = instance[i]     # Valor do atributo na instância atual
        delta = abs(coef * value)
        delta_features.append((feature, delta))


    # Ordena os atributos pela diferença de probabilidade (maior primeiro)
    delta_features.sort(key=lambda x: x[1], reverse=True)

    # Seleciona os atributos que contribuem para a classe prevista
    pi_explanation = []
    threshold = abs(prob_predicted_class[0] - prob_predicted_class[1])  # Diferença de probabilidade entre as duas classes
    current_threshold = 0
    for idx, (feature, delta) in enumerate(delta_features): 
        current_threshold += delta
        pi_explanation.append((feature, instance[features.index(feature)]))
        if current_threshold > threshold:
            break

    return pi_explanation

# Selecionando a primeira instância do conjunto de teste como um array NumPy
instance = X_test.iloc[0].to_numpy()

# Calculando a PI-explicação para a instância
pi_explanation = calculate_pi_explanation_loglinear(instance, clf, list(X.columns))

# Exibindo a PI-explicação
print(f"PI-explicação Log-Linear para a instância {instance}: {pi_explanation}")


PI-explicação Log-Linear para a instância [6.3 2.8 5.1 1.5]: [('petal length (cm)', 5.1)]


c:\Users\gleilson.pedro.SAUDE\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
